This box will set up the name of the slice and set the expected resources<br>

In [40]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

exp_requires = {'core': 10*2, 'nic': 20}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break


import os
slice_name="IP_Test_" + os.getenv('NB_USER')

try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
except:
    print("You don't have a slice named %s yet." % slice_name)
    slice = fablib.new_slice(name=slice_name)

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,sjzhu_0000105651
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


You don't have a slice named IP_Test_sjzhu yet.


<br><br>Next we name and set the networks and interfaces<br>

In [41]:
# this cell sets up the hosts and router
node_names = ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]
for n in node_names:
    node = slice.add_node(name=n, site=site_name, cores=8, ram=32, disk=10, image='default_ubuntu_22')
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")
    
# this cell sets up the network links
nets = [
    {"name": "net1",   "nodes": ["r1","c1","c2"]},
    {"name": "net2",   "nodes": ["r2","p1"]},
    {"name": "net3",   "nodes": ["r3","p2"]},
    {"name": "net4",   "nodes": ["r4","p3","c3"]},
    {"name": "net12",   "nodes": ["r1","r2"]},
    {"name": "net23",   "nodes": ["r2","r3"]},
    {"name": "net34",   "nodes": ["r3","r4"]},
    {"name": "net41",   "nodes": ["r4","r1"]},
]
for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

<br><br>Request the slice and wait until it has been reserved<br>

In [ ]:
slice.submit()


Retry: 1, Time: 42 sec


ID,4a06dca0-f6fa-469c-9b18-e3d68b4f8db7
Name,IP_Test_sjzhu
Lease Expiration (UTC),2023-11-14 05:26:57 +0000
Lease Start (UTC),2023-11-13 05:26:59 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,Configuring


ID,Name,Site,Type,State,Error
53adccf4-ebec-42d6-840d-4843e4e48623,c1,KANS,node,Ticketed,
5389cc65-a294-4724-ad93-f9f71494d749,c2,KANS,node,Ticketed,
ea319786-94b5-49ea-a59e-c7bff99c2c2d,c3,KANS,node,Ticketed,
1b0d78f9-1364-42ca-a9f8-2391986100c6,p1,KANS,node,Ticketed,
1c574ce5-ba40-46be-9f06-af6265b82c6b,p2,KANS,node,Ticketed,
ac0baca8-01d9-4dec-8103-1641de5fab26,p3,KANS,node,Ticketed,
92d772fb-bfec-4a27-855d-91738eb3628f,r1,KANS,node,Ticketed,
17ffca21-ccc4-44a8-9e3b-158ab1c591d5,r2,KANS,node,Ticketed,
eafb2137-ff73-46f6-b971-a79a4ee38c29,r3,KANS,node,Ticketed,
67fa3893-7c2a-459c-a0ea-da7c616b46ca,r4,KANS,node,Ticketed,


In [4]:
slice.wait_ssh(progress=True)

Waiting for slice . Slice state: StableOK
Waiting for ssh in slice . ssh successful


True

In [5]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "c1-net1-p1":  {"addr": "10.0.0.1", "subnet": "10.0.0.0/16"},
    "c2-net1-p1":  {"addr": "10.0.0.2", "subnet": "10.0.0.0/16"},
    "r1-net1-p1":  {"addr": "10.0.1.1", "subnet": "10.0.0.0/16"},
    
    "p1-net2-p1":  {"addr": "20.0.2.1", "subnet": "20.0.0.0/16"},
    "r2-net2-p1":  {"addr": "20.0.1.2", "subnet": "20.0.0.0/16"},
    
    "p2-net3-p1":  {"addr": "30.0.2.2", "subnet": "30.0.0.0/16"},
    "r3-net3-p1":  {"addr": "30.0.1.3", "subnet": "30.0.0.0/16"},
    
    "p3-net4-p1":  {"addr": "40.0.2.3", "subnet": "40.0.0.0/16"},
    "c3-net4-p1":  {"addr": "40.0.0.3", "subnet": "40.0.0.0/16"},
    "r4-net4-p1":  {"addr": "40.0.1.4", "subnet": "40.0.0.0/16"},
    
    "r1-net12-p1":  {"addr": "12.0.1.1", "subnet": "12.0.0.0/16"},
    "r2-net12-p1":  {"addr": "12.0.1.2", "subnet": "12.0.0.0/16"},
    
    "r2-net23-p1":  {"addr": "23.0.1.2", "subnet": "23.0.0.0/16"},
    "r3-net23-p1":  {"addr": "23.0.1.3", "subnet": "23.0.0.0/16"},
    
    "r3-net34-p1":  {"addr": "34.0.1.3", "subnet": "34.0.0.0/16"},
    "r4-net34-p1":  {"addr": "34.0.1.4", "subnet": "34.0.0.0/16"},
    
    "r4-net41-p1":  {"addr": "41.0.1.4", "subnet": "41.0.0.0/16"},
    "r1-net41-p1":  {"addr": "41.0.1.1", "subnet": "41.0.0.0/16"},
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))
    
for n in ['r1','r2','r3','r4']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")
    
for iface in slice.get_interfaces():
    iface.ip_link_up()
    

rt_conf = [
    {"name": "r1",  "addr": "20.0.0.0/16", "gw": "12.0.1.2"},
    {"name": "r1",  "addr": "40.0.0.0/16", "gw": "41.0.1.4"},
    {"name": "r1",  "addr": "30.0.0.0/16", "gw": "12.0.1.2"},
    
    {"name": "r2",  "addr": "10.0.0.0/16", "gw": "12.0.1.1"},
    {"name": "r2",  "addr": "30.0.0.0/16", "gw": "23.0.1.3"},
    {"name": "r2",  "addr": "40.0.0.0/16", "gw": "23.0.1.3"},
    
    {"name": "r3",  "addr": "20.0.0.0/16", "gw": "23.0.1.2"},
    {"name": "r3",  "addr": "40.0.0.0/16", "gw": "34.0.1.4"},
    {"name": "r3",  "addr": "10.0.0.0/16", "gw": "34.0.1.4"},
    
    {"name": "r4",  "addr": "10.0.0.0/16", "gw": "41.0.1.1"},
    {"name": "r4",  "addr": "30.0.0.0/16", "gw": "34.0.1.3"},
    {"name": "r4",  "addr": "20.0.0.0/16", "gw": "34.0.1.3"}
    
]

for rt in rt_conf:
    slice.get_node(name=rt['name']).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])
    
    
for n in ["c1","c2","c3","p1","p2","p3", "r1", "r2", "r3","r4"]:
    slice.get_node(name=n).execute("sudo apt update; sudo apt -y install net-tools", quiet=True)
    
for n in ["c1","c2"]:
    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp6s0", quiet=False)
for n in ["p1"]:
    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp6s0", quiet=False)
for n in ["p2"]:
    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp6s0", quiet=False)
for n in ["c3","p3"]:
    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp6s0", quiet=False)
    
for n in ["r1"]:
    slice.get_node(name=n).execute("sudo route add default gw 40.0.1.4 enp6s0", quiet=False)
for n in ["r2"]:
    slice.get_node(name=n).execute("sudo route add default gw 10.0.1.1 enp7s0", quiet=False)
for n in ["r3"]:
    slice.get_node(name=n).execute("sudo route add default gw 20.0.1.2 enp7s0", quiet=False)
for n in ["r4"]:
    slice.get_node(name=n).execute("sudo route add default gw 30.0.1.3 enp6s0", quiet=False)
    
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
net.ipv4.ip_forward = 1
 SIOCADDRT: Network is unreachable
  SIOCADDRT: Network is unreachable
  SIOCADDRT: Network is unreachable
  SIOCADDRT: Network is unreachable
 

,SSH command
Name,
c1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe9f:9e19
c2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:feba:497d
c3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe5e:2c46
p1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:febe:fb12
p2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:febb:8b6a
p3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe5a:5a17
r1,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe4d:732d
r2,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fece:bbfa
r3,ssh -i /home/fabric/work/fabric_config/slice_key -J sjzhu_0000105651@bastion.fabric-testbed.net -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe50:37f0


Test a few connections through the network

In [6]:
slice.get_node(name="c1").execute("ping 30.0.2.2 -c 5", quiet=False)

PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.
64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=2.30 ms
64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.317 ms
64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.282 ms
64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.291 ms
64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.361 ms

--- 30.0.2.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4059ms
rtt min/avg/max/mdev = 0.282/0.710/2.303/0.796 ms


('PING 30.0.2.2 (30.0.2.2) 56(84) bytes of data.\n64 bytes from 30.0.2.2: icmp_seq=1 ttl=61 time=2.30 ms\n64 bytes from 30.0.2.2: icmp_seq=2 ttl=61 time=0.317 ms\n64 bytes from 30.0.2.2: icmp_seq=3 ttl=61 time=0.282 ms\n64 bytes from 30.0.2.2: icmp_seq=4 ttl=61 time=0.291 ms\n64 bytes from 30.0.2.2: icmp_seq=5 ttl=61 time=0.361 ms\n\n--- 30.0.2.2 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4059ms\nrtt min/avg/max/mdev = 0.282/0.710/2.303/0.796 ms\n',
 '')

In [7]:
slice.get_node(name="p1").execute("ping 40.0.2.3 -c 5", quiet=False)

PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.
64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=1.24 ms
64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.345 ms
64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.310 ms
64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.306 ms
64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.301 ms

--- 40.0.2.3 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4067ms
rtt min/avg/max/mdev = 0.301/0.500/1.242/0.370 ms


('PING 40.0.2.3 (40.0.2.3) 56(84) bytes of data.\n64 bytes from 40.0.2.3: icmp_seq=1 ttl=61 time=1.24 ms\n64 bytes from 40.0.2.3: icmp_seq=2 ttl=61 time=0.345 ms\n64 bytes from 40.0.2.3: icmp_seq=3 ttl=61 time=0.310 ms\n64 bytes from 40.0.2.3: icmp_seq=4 ttl=61 time=0.306 ms\n64 bytes from 40.0.2.3: icmp_seq=5 ttl=61 time=0.301 ms\n\n--- 40.0.2.3 ping statistics ---\n5 packets transmitted, 5 received, 0% packet loss, time 4067ms\nrtt min/avg/max/mdev = 0.301/0.500/1.242/0.370 ms\n',
 '')

Use this to extend the slice when needed

In [8]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

slice.update()
_ = slice.show()

ID,9bb986ed-c3f2-489b-a08a-513e323b4bdc
Name,IP_Test_sjzhu
Lease Expiration (UTC),2023-11-27 03:38:03 +0000
Lease Start (UTC),2023-11-13 02:59:56 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


Use this section to delete the slice when needed

In [39]:
slice.delete()


In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()

Used for temp work 

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()
import os
slice_name="Project_Test_" + os.getenv('NB_USER')

slice = fablib.get_slice(slice_name)

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)

# Manual tweaking to limit node label overlap in the visualization
#pos["p2"] += (0.08, 0.0)
#pos["net3"] += (0.08, 0.0)


nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

NDN Setup:

In [37]:
hosts = node_names

FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}

def make_fs_activate_json(node):
    activate_json = {
        'eal': {
            'memPerNuma': {'0': 4*1024},
            'filePrefix': 'producer',
        },
        'mempool': {
            'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
            'INDIRECT': {'capacity': 2**16-1},
            'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
        },
        'face': {
            'scheme': 'memif',
            'socketName': MEMIF_ARGS['socketName'],
            'id': MEMIF_ARGS['id'],
            'dataroom': 9000,
            'role': 'client',
        },
        'fileServer': {
            'mounts': [
                {'prefix': f"/{node}/usr-local-share", 'path': "/usr/local/share"}
            ],
            'segmentLen': 6 * 1024,
        },
    }
    return activate_json

net_to_nodes = {
    "net1": ["r1","c1","c2"],
    "net2": ["r2","p1"],
    "net3": ["r3","p2"],
    "net4": ["r4","p3","c3"],
    "net12": ["r1","r2"],
    "net23": ["r2","r3"],
    "net34": ["r3","r4"],
    "net41": ["r4","r1"],
}

adj_list = {
    "r1": ["c1","c2","r2","r4"],
    "r2": ["p1","r1","r3"],
    "r3": ["p2","r2","r4"],
    "r4": ["p3","c3","r1","r3"],
    "c1": ["r1"],
    "c2": ["r1"],
    "c3": ["r4"],
    "p1": ["r2"],
    "p2": ["r3"],
    "p3": ["r4"],
}
consumers = ["c1","c2","c3"]
routers = ["r1","r2","r3","r4"]
producers = ["p1", "p2", "p3"]

In [20]:
forwarder_hashes = {}
face_hashes = {}
node_net_interface = {}

In [21]:
import json
interfaces = json.loads(slice.list_interfaces(output="json"))
for i in interfaces:
    h = i["node"]
    net = i["network"]
    inf = i["physical_dev"]
    try:
        node_net_interface[h]
    except KeyError:
        node_net_interface[h] = {}
    node_net_interface[h][net] = inf
print(node_net_interface)

[
    {
        "name": "c1-net1-p1",
        "short_name": "p1",
        "node": "c1",
        "network": "net1",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "0A:52:8F:34:96:C6",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "10.0.0.1",
        "numa": "6"
    },
    {
        "name": "c2-net1-p1",
        "short_name": "p1",
        "node": "c2",
        "network": "net1",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "0A:56:AB:E2:68:40",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "10.0.0.2",
        "numa": "6"
    },
    {
        "name": "c3-net4-p1",
        "short_name": "p1",
        "node": "c3",
        "network": "net4",
        "bandwidth": "100",
        "mode": "config",
        "vlan": "",
        "mac": "16:23:F2:AD:26:F0",
        "physical_dev": "enp6s0",
        "dev": "enp6s0",
        "ip_addr": "40.0.0.3",
     

In [23]:
import shlex
import json

# add eth ports to each node
for h in (consumers + producers):
    node = slice.get_node(h)
    # consumers and producers only have 1 interface
    inf = next(iter(node_net_interface[h].values()))
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

true
{"id":"ACD2H53TMUI36","isDown":false,"macAddr":"0a:52:8f:34:96:c6","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"GVL1D79HNC4M8","isDown":false,"macAddr":"0a:56:ab:e2:68:40","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"HLLQQ4KG7NG6G","isDown":false,"macAddr":"16:23:f2:ad:26:f0","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"HUTK14PMB1LE6","isDown":false,"macAddr":"12:6c:f4:55:67:15","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"KAO1QT16OVJ3M","isDown":false,"macAddr":"1a:7c:39:4f:e8:1c","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"0DUK5I1PE24BG","isDown":false,"macAddr":"1e:51:b3:db:01:58","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}


ValueError: too many values to unpack (expected 2)

In [25]:
for h in routers:
    forwarder_hashes[h] = {}
    for net, inf in iter(node_net_interface[h].items()):
        node = slice.get_node(h)
        output = node.execute(f'''
            echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
            ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif {inf} --xdp --mtu 1500
            ''')
        forwarder_hashes[h][net] = json.loads(output[0].split("\n")[-2])

true
{"id":"OJ3TV04CG8L4E","isDown":false,"macAddr":"22:6a:06:25:64:06","mtu":1500,"name":"net_af_xdp_4","numaSocket":null}
{"id":"OJ3TV04CG8L4C","isDown":false,"macAddr":"26:39:f6:dc:cb:ce","mtu":1500,"name":"net_af_xdp_5","numaSocket":null}
 2023/11/13 04:50:48 graphql: ndndpdk-svc is already activated
 {"id":"OJ3TV04CG8L4A","isDown":false,"macAddr":"1e:a6:b7:27:c2:17","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
 2023/11/13 04:50:50 graphql: ndndpdk-svc is already activated
 true
{"id":"U51QJCCGBG414","isDown":false,"macAddr":"26:ec:72:3c:58:74","mtu":1500,"name":"net_af_xdp_5","numaSocket":null}
{"id":"U51QJCCGBG416","isDown":false,"macAddr":"26:9a:53:ec:da:62","mtu":1500,"name":"net_af_xdp_4","numaSocket":null}
 2023/11/13 04:50:55 graphql: ndndpdk-svc is already activated
 {"id":"U51QJCCGBG410","isDown":false,"macAddr":"26:3e:26:77:3b:09","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
 2023/11/13 04:50:57 graphql: ndndpdk-svc is already activated
 true
{"id":"8DQ0TTHA

In [26]:
print(forwarder_hashes)

{'c1': {'id': 'ACD2H53TMUI36', 'isDown': False, 'macAddr': '0a:52:8f:34:96:c6', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c2': {'id': 'GVL1D79HNC4M8', 'isDown': False, 'macAddr': '0a:56:ab:e2:68:40', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'c3': {'id': 'HLLQQ4KG7NG6G', 'isDown': False, 'macAddr': '16:23:f2:ad:26:f0', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p1': {'id': 'HUTK14PMB1LE6', 'isDown': False, 'macAddr': '12:6c:f4:55:67:15', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p2': {'id': 'KAO1QT16OVJ3M', 'isDown': False, 'macAddr': '1a:7c:39:4f:e8:1c', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'p3': {'id': '0DUK5I1PE24BG', 'isDown': False, 'macAddr': '1e:51:b3:db:01:58', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'r1': {'net41': {'id': 'OJ3TV04CG8L4E', 'isDown': False, 'macAddr': '22:6a:06:25:64:06', 'mtu': 1500, 'name': 'net_af_xdp_4', 'numaSocket': None}, 'net12': {'id': 'OJ3TV04CG8L4C

In [27]:
# add interfaces for consumers and producers
for h in (consumers + producers):
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(h)
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

KeyError: 'macAddr'

In [35]:
# this is a janky for loop that uses the routers to set up all the ethernet faces
for h in routers:
    node = slice.get_node(h)
    for net, output in iter(forwarder_hashes[h].items()):
        local_mac = output['macAddr']
        for r in net_to_nodes[net]:
            if h == r:
                continue
            else:
                print(f"h: {h}, r: {r}, net: {net}")
                print(f"local_mac: {local_mac}")
                if r in routers:
                    remote_mac = forwarder_hashes[r][net]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                else:
                    remote_mac = forwarder_hashes[r]['macAddr']
                    print(f"remote_mac: {remote_mac}")
                    # do the reverse since we have the correct mac address here
                    remote_node = slice.get_node(r)
                    out2 = remote_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {remote_mac} --remote {local_mac}")
                    face_hashes[f"{r}:{h}"] = json.loads(out2[0])
                # we only do 1 way for routers because we're going to do the other side in another loop
                out = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
                face_hashes[f"{h}:{r}"] = json.loads(out[0])

h: r1, r: r4, net: net41
local_mac: 22:6a:06:25:64:06
remote_mac: 02:dc:a4:ab:d6:d8
{"id":"OV9D98EJO0JKAQ6H"}
h: r1, r: r2, net: net12
local_mac: 26:39:f6:dc:cb:ce
remote_mac: 26:ec:72:3c:58:74
{"id":"OV9D98EJOSG4EPUT"}
h: r1, r: c1, net: net1
local_mac: 1e:a6:b7:27:c2:17
remote_mac: 0a:52:8f:34:96:c6
{"id":"A07I7D92UEN3884N"}
{"id":"OV9D98EJO4HKEQEH"}
h: r1, r: c2, net: net1
local_mac: 1e:a6:b7:27:c2:17
remote_mac: 0a:56:ab:e2:68:40
{"id":"GJVHRF3EV806PQ8"}
{"id":"OV9D98EJO0IKSQEL"}
h: r2, r: r1, net: net12
local_mac: 26:ec:72:3c:58:74
remote_mac: 26:39:f6:dc:cb:ce
{"id":"U9BA546F3S31EBS9"}
h: r2, r: p1, net: net2
local_mac: 26:9a:53:ec:da:62
remote_mac: 12:6c:f4:55:67:15
{"id":"HIN4NCJ93LIU7OAA"}
{"id":"U9BA546F3C0168KF"}
h: r2, r: r3, net: net23
local_mac: 26:3e:26:77:3b:09
remote_mac: 02:88:da:4f:7a:e8
{"id":"U9BA546F3C2H8BCD"}
h: r3, r: p2, net: net3
local_mac: 02:da:d0:de:84:4d
remote_mac: 1a:7c:39:4f:e8:1c
{"id":"K6IHCLBPGBLJS0OM"}
{"id":"81GGBLRL4E82DCD9"}
h: r3, r: r2, net: ne

In [38]:
for p in producers:
    producer_node = slice.get_node(p)
    output = producer_node.execute(f'''
        MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh $(echo $MEMIF_FACE | jq -r .id)
        ''')
    # start fileserver instance
    output = producer_node.execute(f'''
        sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
        echo {shlex.quote(json.dumps(make_fs_activate_json(p)))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
        ''')

Incorrect Usage: flag needs an argument: -nh

NAME:
   ndndpdk-ctrl insert-fib - Insert or replace a FIB entry

USAGE:
 2023/11/13 05:24:09 graphql: createFace is disallowed; is NDN-DPDK forwarder activated?
    ndndpdk-ctrl insert-fib command [command options] [arguments...]

COMMANDS:
   help, h  Shows a list of commands or help for one command

OPTIONS:
   --name prefix  name prefix
   --nh ID        FIB nexthop face ID (repeatable)
   --strategy ID  forwarding strategy ID
   --params JSON  forwarding strategy parameters JSON
   --help, -h     show help
 2023/11/13 05:24:09 flag needs an argument: -nh
  2023/11/13 05:24:11 graphql: error creating face rte_eth_dev_start 1 Operation not permitted
 Incorrect Usage: flag needs an argument: -nh

NAME:
   ndndpdk-ctrl insert-fib - Insert or replace a FIB entry

USAGE:
   ndndpdk-ctrl insert-fib command [command options] [arguments...]

COMMANDS:
   help, h  Shows a list of commands or help for one command

OPTIONS:
   --name prefix  name 

In [25]:
print(face_hashes)

{'r1:c1': {'id': 'TALFD1MM3F9G1GK5'}, 'r1:c2': {'id': 'TALFD1MM3RD09ISA'}, 'r1:r2': {'id': 'TALFD1MM3FFG3IS6'}, 'r1:r4': {'id': 'TALFD1MM3JFG5G40'}, 'r2:p1': {'id': 'IV2U7UAVU7D66A46'}, 'r2:r1': {'id': 'IV2U7UAVUBF6IAS6'}, 'r2:r3': {'id': 'IV2U7UAVUVFM69KB'}, 'r3:p2': {'id': 'EOJFDVRLHKB3VJ45'}, 'r3:r2': {'id': 'EOJFDVRLHC93VJK1'}, 'r3:r4': {'id': 'EOJFDVRLHCAJTISC'}, 'r4:p3': {'id': 'M4RQDU4HRBVBTPCO'}, 'r4:c3': {'id': 'M4RQDU4HRJVBPQSU'}, 'r4:r1': {'id': 'M4RQDU4HRFVBJP4M'}, 'r4:r3': {'id': 'M4RQDU4HRBQBLOKM'}, 'c1:r1': {'id': 'HHPD8OIKVR274VR4'}, 'c2:r1': {'id': '2HBGQ5KSU7GG8E7A'}, 'c3:r4': {'id': 'NTBPP1M56KV9L26N'}, 'p1:r2': {'id': '1BC8M6I0QQA7QCLV'}, 'p2:r3': {'id': 'AVHPVIITCIMO1RRD'}, 'p3:r4': {'id': 'QA6MD80PSSI67K72'}}


In [28]:
# add FIB entries to point to producer
for p in producers:
    for r in adj_list[p]:
        c_node = slice.get_node(r)
        c_to_r = face_hashes[f"{r}:{p}"]
        print(c_to_r)
        output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /{p} --nh {c_to_r['id']}")

{'id': 'IV2U7UAVU7D66A46'}
{"id":"IV6U5M8BMUE2G9DM80DSK"}
{'id': 'EOJFDVRLHKB3VJ45'}
{"id":"EONFFNP1P987FGLSNG56E"}
{'id': 'M4RQDU4HRBVBTPCO'}
{"id":"M4VQFM65JIQF3QD626D0I"}
